# Correspondence-driven plane-based M3C2 (PB_M3C2) with externally produced similar regions

In [ ]:
import py4dgeo
import numpy as np

In this notebook, we present a variant of the *Correspondence-driven plane-based M3C2* (PB-M3C2, [Zahs et al., 2022](#References)) algorithm. In contrast to the [base algorithm](PB_M3C2.ipynb), we are taking an alternative input source: Externally produced pairs of coordinates which should correspond to regions of similarity. It is not required to do this based on the segment IDs (as was done in [this variant of the algorithm](PB_M3C2_scenario2.ipynb)) - instead similar regions are matched with segments based on proximity.

The initial setup of the algorithm is exactly like we have seen before:

In [ ]:
py4dgeo.set_interactive_backend("vtk")

In [ ]:
py4dgeo.ensure_test_data_availability()

In [ ]:
epoch0, epoch1 = py4dgeo.read_from_xyz(
    "plane_horizontal_t1.xyz", "plane_horizontal_t2.xyz"
)

In [ ]:
alg = py4dgeo.PB_M3C2()

In [ ]:
xyz_epoch0, xyz_epoch1, segment_id = alg.export_segmented_point_cloud_and_segments(
    epoch0=epoch0,
    epoch1=epoch1,
)

We now need to load our externally produced pairs of proximity regions. We are expecting a `numpy` array of seven columns and one row per proximity pair. The first three columns denote the XYZ coordinates of the center of mass in the first epoch. Likewise, the forth to sixth column denote the center of mass in the second epoch. The last column can contain either `0` or `1` depending on whether the two regions contain segments that look alike. In this example, we store the data as a CSV file, but you are free to choose your own format, as long as you are able to read it into a `np.array` data structure:

In [ ]:
pairs_of_points = np.loadtxt(
    py4dgeo.find_file("testdata-proximity-labelling.csv"), delimiter=","
)

Given this data, we can perform the proximity matching using the following function. The `threshold_max_distance` parameter controls the proximity matching between given regions and found segments:

In [ ]:
extended_y = py4dgeo.generate_extended_y_from_prior_knowledge(
    segments=segment_id, pairs_of_points=pairs_of_points, threshold_max_distance=5
)

We are now ready to train the algorithm and do distance calculation:

In [ ]:
alg.training(
    extracted_segments_file_name="extracted_segments.seg", extended_y=extended_y
)

In [ ]:
distances, uncertainties = alg.compute_distances(epoch0=epoch0, epoch1=epoch1)

### References

* Zahs, V., Winiwarter, L., Anders, K., Williams, J.G., Rutzinger, M. & Höfle, B. (2022): Correspondence-driven plane-based M3C2 for lower uncertainty in 3D topographic change quantification. ISPRS Journal of Photogrammetry and Remote Sensing, 183, pp. 541-559. DOI: [10.1016/j.isprsjprs.2021.11.018](https://doi.org/10.1016/j.isprsjprs.2021.11.018).